In [ ]:
import os
import pathlib
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
from PIL import Image, ImageDraw

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define dataset directories
dataset_dir = pathlib.Path("/content/drive/MyDrive/weatherimages")

train_dir = dataset_dir / "train"
test_dir = dataset_dir / "test"

# Create necessary directories for the dataset
classes = ['sunny', 'rainy', 'cloudy', 'snowy']

# Create directories if they don't exist
for split in [train_dir, test_dir]:
    for cls in classes:
        os.makedirs(split / cls, exist_ok=True)

# Function to create a sample image
def create_image(save_path, color):
    img = Image.new('RGB', (400, 400), color=color)
    draw = ImageDraw.Draw(img)
    draw.text((200, 200), color, fill='black')
    img.save(save_path)

# Generate images for each class
colors = {'sunny': 'yellow', 'rainy': 'blue', 'cloudy': 'gray', 'snowy': 'white'}

for cls, color in colors.items():
    for i in range(10):
        create_image(train_dir / cls / f'{cls}_{i}.jpg', color)
        create_image(test_dir / cls / f'{cls}_{i}.jpg', color)

# Define image size and batch size
image_size = (400, 400)
batch_size = 50

# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1.0/255
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Define model architecture
model = models.Sequential([
    layers.Input(shape=(400, 400, 3)),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(3),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(3),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(2),
    layers.Conv2D(128, 3, activation='relu'),
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Added dropout for regularization
    layers.Dense(len(classes), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks for early stopping, learning rate scheduling, and model checkpointing
callbacks = [
    keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=2),
    keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True),
]

# Train the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=test_generator,
    callbacks=callbacks
)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}")


Mounted at /content/drive
Found 40 images belonging to 4 classes.
Found 40 images belonging to 4 classes.
Epoch 1/20
1/1 [==============================] - 22s 22s/step - loss: 1.3805 - accuracy: 0.2250 - val_loss: 1.3252 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 2/20


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 18s 18s/step - loss: 1.3371 - accuracy: 0.6000 - val_loss: 1.2668 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 3/20
1/1 [==============================] - 17s 17s/step - loss: 1.2752 - accuracy: 0.5750 - val_loss: 1.1448 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 4/20
1/1 [==============================] - 18s 18s/step - loss: 1.1564 - accuracy: 0.7250 - val_loss: 0.9914 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 5/20
1/1 [==============================] - 18s 18s/step - loss: 1.0062 - accuracy: 0.7000 - val_loss: 0.8229 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 6/20
1/1 [==============================] - 17s 17s/step - loss: 0.9406 - accuracy: 0.7250 - val_loss: 0.6689 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 7/20
1/1 [==============================] - 17s 17s/step - loss: 0.7202 - accuracy: 0.7250 - val_loss: 0.5545 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 8/20
1/1 [==============================] - 17s 17s/step - loss: 0.5707 - accuracy: 0.7500 - 